In [1]:
import os

In [2]:
%pwd

'd:\\Learning Phase(2023-2024)\\Resume_projects\\Brief-A-Text\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Learning Phase(2023-2024)\\Resume_projects\\Brief-A-Text'

# Entity

In [1]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Configuration Manager

In [11]:
from briefAtext.constant import *
from src.briefAtext.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


# Components

In [14]:
import os
import urllib.request as request
import zipfile
from briefAtext.logging import logger
from briefAtext.utils.common import get_size

In [16]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-07-06 17:58:26,312: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-06 17:58:26,314: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-06 17:58:26,317: INFO: common: Created directory at: artifacts]
[2024-07-06 17:58:26,319: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-07-06 17:58:29,342: INFO: 3911023062: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 4176912
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3a33dbed522ef282530af1a2b5474287ea29e9ee0cb3cff5babaf3220425c4dc"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 662F:D99F5:9A9DD:D64F4:6689386C
Accept-Ranges: bytes
Date: Sat, 06 Jul 2024 12:28:28 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100030-HYD
X-Cache: MI